In [24]:
import pandas as pd
import os
from dotenv import load_dotenv
import warnings
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
import traceback  # 상세 에러 메시지 출력을 위해 추가

# ✅ 환경 변수 로드 및 설정
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings('ignore')
load_dotenv()

# ✅ 결과 저장을 위한 객체 선언
results = {}
list_df = {}

# ✅ 데이터 로드: ../data 경로의 모든 pkl 파일 읽기
data_path = os.path.join('..', 'data')
for file in os.listdir(data_path):
    if file.endswith('.pkl'):
        file_path = os.path.join(data_path, file)
        df_name = file.replace('.pkl', '')
        list_df[df_name] = pd.read_pickle(file_path)

# ✅ globals()에 데이터프레임 등록 (해결책 1)
for name, df in list_df.items():
    globals()[name] = df  # ✅ `analyze_multiple_dataframes`에서 접근 가능하도록 등록

list_df_text = ", ".join(list_df.keys())

# ✅ 프롬프트 및 함수 코드 불러오기
prompt = open(f'prompt/001_prompt_data_summary.txt', 'r', encoding='utf-8').read().format(list_df_text=list_df_text)
func_code = open(f'sample_func/func_data_summary.py', 'r', encoding='utf-8').read()

# ✅ LLM에 첫 번째 코드 요청 (데이터 구조 분석 단계)
chain = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "### 참고 코드:\n{func_code}\n\n"),
    ("user", "### list_df:\n{list_df_text}\n\n")
]) | llm

response = chain.invoke({"func_code": func_code, "list_df_text": list_df_text}).content

attempt_count = 0  # 실행 시도 횟수
success = False  # 코드 실행 성공 여부

while attempt_count < 2:  # 최초 실행 + 1회 재시도 가능
    try:
        if "```python" in response:
            modified_code = response.split("```python")[-1].split("```")[0]
            print(f"[LOG] 실행할 코드:\n{modified_code}")
            exec(modified_code, globals())  # ✅ LLM이 생성한 코드 실행
            print(f"[Stage 1. 데이터 구조 파악] 1차 시도 | ✅ 성공적으로 실행되었습니다!")
            success = True
            break  # 실행 성공 시 루프 종료
    except Exception as e:
        error_trace = traceback.format_exc()  # ✅ 에러 상세 정보 수집
        print(f"❌ {attempt_count+1}차 시도 : LLM 생성 코드 실행 중 오류 발생\n{error_trace}")

        if attempt_count == 0:  # 최초 실행 실패 시 1회만 재생성
            print("🔄 오류 메시지를 기반으로 코드 수정 요청 중...")

            # ✅ LLM에 더 명확한 오류 메시지 전달 (해결책 2)
            prompt_error_fix = f"""
            ### 코드 수정 요청

            이전 코드 실행 중 다음 오류가 발생했습니다:
            ```
            {error_trace}
            ```

            위 오류를 해결한 새로운 코드를 생성하세요.
            - 기존 코드에서 오류를 수정한 버전으로 제공해야 합니다.
            - 오류 원인을 분석하여 반드시 실행 가능하도록 보완해야 합니다.
            - 필요한 경우, 추가적인 데이터 핸들링 코드를 포함해야 합니다.

            ```python
            # 필요한 코드 삽입
            ```
            """
            chain_error_fix = ChatPromptTemplate.from_messages([
                ("system", prompt_error_fix),
                ("user", "### 기존 코드:\n{modified_code}\n\n")
            ]) | llm

            response = chain_error_fix.invoke({"modified_code": modified_code}).content
        else:
            print("❌ 코드 실행 최종적으로 실패. 프로세스를 중단합니다.")

        attempt_count += 1  # 재시도 횟수 증가

# ✅ Stage 1 결과 파일 로드 (엑셀)
stage1_path = "../output/stage1/eda_summary.xlsx"
if os.path.exists(stage1_path):
    stage1_results = pd.read_excel(stage1_path, sheet_name=0)  # 첫 번째 시트 사용
    results["stage1"] = stage1_results  # ✅ Stage 1 결과 저장
else:
    print("❌ Stage 1 결과 파일이 존재하지 않습니다.")
    raise FileNotFoundError("Stage 1 결과 파일을 찾을 수 없습니다.")

# ✅ RAG 기반 컬럼 설명 추가
def load_vectorstore():
    if os.path.exists("./vectordb"):
        embeddings = OpenAIEmbeddings(
            model="text-embedding-3-large",
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        try:
            return FAISS.load_local("./vectordb", embeddings, allow_dangerous_deserialization=True)
        except Exception as e:
            print(f"⚠️ 벡터스토어 로드 중 오류 발생: {e}")
            return None
    return None

vectorstore = load_vectorstore()

def search_column_descriptions(col_desc_df, vectorstore):
    """RAG를 사용하여 컬럼 설명을 검색하는 함수"""
    data = []
    df_after_llm = col_desc_df[['데이터프레임명', '컬럼명']]
    
    for i, row in df_after_llm.iterrows():
        table_name = row['데이터프레임명']
        col = row['컬럼명']
        
        search_query = f"테이블명 : {table_name} | 컬럼명 : {col}"
        docs = vectorstore.similarity_search(search_query, k=1)
        
        if docs:
            best_match = docs[0].page_content
            data.append({
                '데이터프레임명': table_name,
                '컬럼명': col,
                '컬럼설명': best_match.split('\n')[3].split('설명')[1].strip()
            })
        else:
            data.append({
                '데이터프레임명': table_name,
                '컬럼명': col,
                '컬럼설명': "설명 없음"
            })
    
    return pd.DataFrame(data)

# ✅ Stage 2: RAG 기반 컬럼 설명 추가
rag_results = search_column_descriptions(stage1_results, vectorstore)

# ✅ Stage 1 결과와 RAG 결과를 통합
final_df = pd.merge(
    stage1_results,  # Stage 1 결과 (데이터 구조 정보)
    rag_results,     # RAG 검색 결과 (컬럼 설명)
    on=['데이터프레임명', '컬럼명'],
    how='left'
)

# ✅ 최종 결과를 메모리 내 저장
results["stage2"] = final_df

print("✅ 모든 단계가 성공적으로 완료되었습니다.")


[LOG] 실행할 코드:

import pandas as pd
import numpy as np

def summarize_data(df, df_name):
    """
    주어진 데이터프레임(df)에 대한 EDA(탐색적 데이터 분석) 결과를 정리하여 반환하는 함수.
    - "컬럼 개요"에 범주형 변수 분포 정보 포함
    - 연속형 변수의 인스턴스(예제) 제외 (토큰 수 절감 목적)
    """

    # ✅ 1. 이상치 탐지 (IQR 방식)
    outliers_info = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()

        if outlier_count > 0:
            outliers_info[col] = int(outlier_count)

    # ✅ 2. 컬럼 개요 (기본 통계 및 결측 정보 + 인스턴스 예제 + 범주형 분포 추가)
    columns_info = []
    for col in df.columns:
        col_dtype = df[col].dtype

        col_info = {
            "데이터프레임명": df_name,
            "컬럼명": col,
            "데이터 타입": col_dtype,
            "count": df[col].count(),
            "mean": round(df[col].m

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
import warnings

# ✅ LangGraph 및 LangChain 관련 모듈
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# ✅ AI Assistant LangGraph Class Import
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings('ignore')
load_dotenv()  # 환경 변수 로드


# ✅ OpenAI API Key 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0,)

# llm 결과 코드 실행 결과가 담길 객체 dict 선언
results = {}
list_df = {}

# ../data 경로의 모든 pkl 파일 읽기
data_path = os.path.join('..', 'data')
for file in os.listdir(data_path):
    if file.endswith('.pkl'):
        file_path = os.path.join(data_path, file)
        df_name = file.replace('.pkl', '')
        list_df[df_name] = pd.read_pickle(file_path)

list_df_text = ", ".join(list_df)

# 프롬프트 및 함수 코드 불러오기
prompt = open(f'prompt/001_prompt_data_summary.txt', 'r', encoding='utf-8').read().format(list_df_text=list_df_text)
func_code = open(f'sample_func/func_data_summary.py', 'r', encoding='utf-8').read()

# LLM에 첫 번째 코드 요청
chain = ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("user", "### 참고 코드:\n{func_code}\n\n"),
            ("user", "### list_df:\n{list_df}\n\n")
        ]) | llm
response = chain.invoke({"func_code": func_code, "list_df": list_df_text}).content

attempt_count = 0  # 실행 시도 횟수
success = False  # 코드 실행 성공 여부

while attempt_count < 2:  # 최초 실행 + 1회 재생성 가능
    try:
        if "```python" in response:
            modified_code = response.split("```python")[-1].split("```")[0]
            print(f"[LOG] 실행할 코드:\n{modified_code}")
            exec(modified_code, globals())  # LLM이 생성한 코드 실행
            print(f"[Stage 1. 데이터 구조 파악] 1차 시도 | ✅ 데이터마트에 대한 구조 파악 코드가 성공적으로 실행되었습니다!")
            success = True
            break  # 실행 성공 시 루프 종료
    except Exception as e:
        error_message = str(e)
        print(f"❌ {attempt_count+1}차 시도 : LLM 생성 코드 실행 중 오류 발생: {error_message}")

        if attempt_count == 0:  # 최초 실행 실패 시 1회만 재생성
            print("🔄 오류 메시지를 기반으로 코드 수정 요청 중...")

            # 오류 메시지를 기반으로 LLM에 코드 수정 요청
            prompt_error_fix = f"""
            ### 코드 수정 요청

            이전 코드 실행 중 다음 오류가 발생했습니다:
            ```
            {error_message}
            ```

            위 오류를 해결한 새로운 코드를 생성하세요.
            - 기존 코드에서 오류를 수정한 버전으로 제공해야 합니다.
            - 코드 실행 가능하도록 보완해야 합니다.

            ```python
            # 필요한 코드 삽입
            ```
            """
            chain_error_fix = ChatPromptTemplate.from_messages([
                        ("system", prompt_error_fix),
                        ("user", "### 기존 코드:\n{modified_code}\n\n")
                    ]) | llm

            response = chain_error_fix.invoke({"modified_code": modified_code}).content
            # print(f'[LOG] 수정된 코드:\n{response}')
        else:
            print("❌ 코드 실행 최종적으로 실패. 프로세스를 중단합니다.")

        attempt_count += 1  # 재시도 횟수 증가

# ✅ FAISS 벡터스토어 로드
def load_vectorstore():
    if os.path.exists("./vectordb"):
        embeddings = OpenAIEmbeddings(
            model="text-embedding-3-large",
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        try:
            return FAISS.load_local("./vectordb", embeddings, allow_dangerous_deserialization=True)
        except Exception as e:
            print(f"벡터스토어 로드 중 오류 발생: {e}")
            return None
    else:
        return None
    
vectorstore = load_vectorstore()
def load_column_descriptions(file_path):
    """컬럼 설명 데이터를 로드하는 함수"""
    if os.path.exists(file_path):
        # Stage 1의 결과에서 첫 번째 시트(데이터 구조 요약) 로드
        return pd.read_excel(file_path, sheet_name=0)
    print("⚠️ 컬럼 설명 파일이 존재하지 않습니다. 설명 없이 진행됩니다.")
    return None

def search_column_descriptions(col_desc_df, vectorstore):
    """RAG를 사용하여 컬럼 설명을 검색하는 함수"""
    data = []
    # Stage 1 결과의 필요한 컬럼만 선택
    df_after_llm = col_desc_df[['데이터프레임명', '컬럼명']]
    
    for i, row in df_after_llm.iterrows():
        table_name = row['데이터프레임명']
        col = row['컬럼명']
        
        # 현재 테이블명을 포함한 검색 쿼리 생성
        search_query = f"테이블명 : {table_name} | 컬럼명 : {col}"
        docs = vectorstore.similarity_search(search_query, k=1)
        
        if docs:
            best_match = docs[0].page_content
            data.append({
                '데이터프레임명': table_name,
                '컬럼명': col,
                '컬럼설명': best_match.split('\n')[3].split('설명')[1].strip()
            })
            print(f"🔍 [{i+1}] {search_query} | {best_match.split('설명')[1].strip()}")
        else:
            data.append({
                '데이터프레임명': table_name,
                '컬럼명': col,
                '컬럼설명': "설명 없음"
            })
            print(f"⚠️ 컬럼 '{col}' 설명 없음")
    
    return pd.DataFrame(data)

# 메인 실행 부분
stage1_path = "output/stage1/eda_summary.xlsx"
col_desc_df = load_column_descriptions(stage1_path)

if col_desc_df is not None:
    # RAG 검색 수행
    rag_results = search_column_descriptions(col_desc_df, vectorstore)
    
    # Stage 1의 결과와 RAG 결과를 통합
    final_df = pd.merge(
        col_desc_df,     # Stage 1의 결과 (원본 데이터 구조 정보)
        rag_results,     # RAG 검색 결과 (컬럼 설명)
        on=['데이터프레임명', '컬럼명'],
        how='left'
    )
    
    # 결과 저장
    output_path = "output/stage1/integrated_column_descriptions.xlsx"
    
    # 원본 Stage 1 결과의 모든 시트를 복사하고, 통합 결과를 새로운 시트로 추가
    with pd.ExcelWriter(output_path) as writer:
        # 먼저 Stage 1의 모든 시트를 복사
        original_sheets = pd.read_excel(stage1_path, sheet_name=None)
        for sheet_name, df in original_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # 통합된 결과를 새로운 시트로 추가
        final_df.to_excel(writer, sheet_name='통합_결과', index=False)
    
    print(f"✅ 통합된 결과가 {output_path}에 저장되었습니다.")


[LOG] 실행할 코드:

import pandas as pd
import numpy as np

def summarize_data(df, df_name):
    """
    주어진 데이터프레임(df)에 대한 EDA(탐색적 데이터 분석) 결과를 정리하여 반환하는 함수.
    - "컬럼 개요"에 범주형 변수 분포 정보 포함
    - 연속형 변수의 인스턴스(예제) 제외 (토큰 수 절감 목적)
    """

    # ✅ 1. 이상치 탐지 (IQR 방식)
    outliers_info = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()

        if outlier_count > 0:
            outliers_info[col] = int(outlier_count)

    # ✅ 2. 컬럼 개요 (기본 통계 및 결측 정보 + 인스턴스 예제 + 범주형 분포 추가)
    columns_info = []
    for col in df.columns:
        col_dtype = df[col].dtype

        col_info = {
            "데이터프레임명": df_name,
            "컬럼명": col,
            "데이터 타입": col_dtype,
            "count": df[col].count(),
            "mean": round(df[col].m

🔍 [1] 테이블명 : cust_enroll_history | 컬럼명 : 고객ID | : 개별 고객을 식별하기 위한 고유 식별 번호
🔍 [2] 테이블명 : cust_enroll_history | 컬럼명 : 가입년월일 | : 고객이 보험 상품에 가입한 날짜 (년, 월, 일 단위).
🔍 [3] 테이블명 : cust_enroll_history | 컬럼명 : 가입담보명 | : 고객이 가입한 담보에 대해 설정된 보장 금액.
🔍 [4] 테이블명 : cust_enroll_history | 컬럼명 : 가입담보금액 | : 고객이 가입한 담보에 대해 설정된 보장 금액.
🔍 [5] 테이블명 : cust_intg | 컬럼명 : 고객ID | : 개별 고객을 식별하기 위한 고유 식별 번호
🔍 [6] 테이블명 : cust_intg | 컬럼명 : 나이 | : 고객의 현재 연령
🔍 [7] 테이블명 : cust_intg | 컬럼명 : 성별 | : 고객의 성별 (남/여)
🔍 [8] 테이블명 : cust_intg | 컬럼명 : 수익자여부 | : 고객이 보험 수익자인지 여부
🔍 [9] 테이블명 : cust_intg | 컬럼명 : CB신용평점 | : CB(Credit Bureau) 기준 고객의 신용평점
🔍 [10] 테이블명 : cust_intg | 컬럼명 : CB신용등급 | : CB(Credit Bureau) 기준 고객의 신용등급
🔍 [11] 테이블명 : cust_intg | 컬럼명 : 두낫콜여부 | : 고객이 마케팅 전화 수신을 거부했는지 여부
🔍 [12] 테이블명 : cust_intg | 컬럼명 : 운전코드명 | : 고객의 운전 관련 코드명 (운전면허 여부 등)
🔍 [13] 테이블명 : cust_intg | 컬럼명 : 성별코드 | : 고객 성별을 나타내는 코드 값
🔍 [14] 테이블명 : cust_intg | 컬럼명 : 피보험자여부 | : 고객이 피보험자인지 여부
🔍 [15] 테이블명 : cust_intg | 컬럼명 : 보험연령 | : 보험 가입 시 산정된 고객의 연령
🔍 [16] 테이블명 : 

In [10]:
import pandas as pd
import os
from dotenv import load_dotenv
import warnings

# ✅ LangGraph 및 LangChain 관련 모듈
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings


# ✅ AI Assistant LangGraph Class Import
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings('ignore')
load_dotenv()  # 환경 변수 로드


# ✅ OpenAI API Key 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai_api_key, temperature=0,)

# llm 결과 코드 실행 결과가 담길 객체 dict 선언
results = {}
list_df = {}

# ../data 경로의 모든 pkl 파일 읽기
data_path = os.path.join('..', 'data')
for file in os.listdir(data_path):
    if file.endswith('.pkl'):
        file_path = os.path.join(data_path, file)
        df_name = file.replace('.pkl', '')
        list_df[df_name] = pd.read_pickle(file_path)

list_df_text = ", ".join(list_df)

# 프롬프트 및 함수 코드 불러오기
prompt = open(f'prompt/001_prompt_data_summary.txt', 'r', encoding='utf-8').read().format(list_df_text=list_df_text)
func_code = open(f'sample_func/func_data_summary.py', 'r', encoding='utf-8').read()

# LLM에 첫 번째 코드 요청
chain = ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("user", "### 참고 코드:\n{func_code}\n\n"),
            ("user", "### list_df:\n{list_df}\n\n")
        ]) | llm
response = chain.invoke({"func_code": func_code, "list_df": list_df_text}).content

attempt_count = 0  # 실행 시도 횟수
success = False  # 코드 실행 성공 여부

In [16]:

# ✅ 프롬프트 및 함수 코드 불러오기
prompt = open(f'prompt/001_prompt_data_summary.txt', 'r', encoding='utf-8').read().format(list_df_text=list_df_text)
func_code = open(f'sample_func/func_data_summary.py', 'r', encoding='utf-8').read()
print("✅ 프롬프트 및 함수 코드 로드 완료\n")

print("🤖 LLM에 코드 생성 요청 중...")

# ✅ LLM에 첫 번째 코드 요청
chain = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "### 참고 코드:\n{func_code}\n\n"),
    ("user", "### list_df:\n{list_df}\n\n")
]) | llm

response = chain.invoke({"func_code": func_code, "list_df": list_df_text}).content

✅ 프롬프트 및 함수 코드 로드 완료

🤖 LLM에 코드 생성 요청 중...


In [18]:
modified_code = response.split("```python")[-1].split("```")[0]
print(f"[LOG] 실행할 코드:\n{modified_code}")
exec(modified_code, globals())

[LOG] 실행할 코드:

import pandas as pd
import numpy as np

def summarize_data(df, df_name):
    """
    주어진 데이터프레임(df)에 대한 EDA(탐색적 데이터 분석) 결과를 정리하여 반환하는 함수.
    - "컬럼 개요"에 범주형 변수 분포 정보 포함
    - 연속형 변수의 인스턴스(예제) 제외 (토큰 수 절감 목적)
    """

    # ✅ 1. 이상치 탐지 (IQR 방식)
    outliers_info = {}
    for col in df.select_dtypes(include=[np.number]).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outlier_count = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()

        if outlier_count > 0:
            outliers_info[col] = int(outlier_count)

    # ✅ 2. 컬럼 개요 (기본 통계 및 결측 정보 + 인스턴스 예제 + 범주형 분포 추가)
    columns_info = []
    for col in df.columns:
        col_dtype = df[col].dtype

        col_info = {
            "데이터프레임명": df_name,
            "컬럼명": col,
            "데이터 타입": col_dtype,
            "count": df[col].count(),
            "mean": round(df[col].m

IndexError: At least one sheet must be visible

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import warnings
import traceback
import json

# ✅ LangGraph 및 LangChain 관련 모듈
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# ✅ 환경 설정
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings('ignore')
load_dotenv()

# ✅ OpenAI API Key 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4", openai_api_key=openai_api_key, temperature=0)

# ✅ 결과 저장을 위한 객체 선언
results = {}
list_df = {}

print("🔄 데이터 로드 시작...")
# ✅ 데이터 로드: ../data 경로의 모든 pkl 파일 읽기
data_path = os.path.join('..', 'data')
for file in os.listdir(data_path):
    if file.endswith('.pkl'):
        file_path = os.path.join(data_path, file)
        df_name = file.replace('.pkl', '')
        list_df[df_name] = pd.read_pickle(file_path)
        print(f"✅ '{df_name}' 데이터 로드 완료")

list_df_text = ", ".join(list_df.keys())
print(f"📊 총 {len(list_df)} 개의 데이터프레임 로드 완료\n")

# ✅ 프롬프트 및 함수 코드 불러오기
prompt = open(f'prompt/001_prompt_data_summary.txt', 'r', encoding='utf-8').read().format(list_df_text=list_df_text)
func_code = open(f'sample_func/func_data_summary.py', 'r', encoding='utf-8').read()
print("✅ 프롬프트 및 함수 코드 로드 완료\n")

def get_detailed_error_info(e):
    """에러에 대한 상세 정보를 반환하는 함수"""
    error_type = type(e).__name__
    error_msg = str(e)
    error_traceback = traceback.format_exc()
    
    return {
        "error_type": error_type,
        "error_message": error_msg,
        "traceback": error_traceback,
        "context": {
            "locals": str(locals()),
            "globals": str([k for k in globals().keys() if not k.startswith('_')])
        }
    }

print("🤖 LLM에 코드 생성 요청 중...")

# ✅ LLM에 첫 번째 코드 요청
chain = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ("user", "### 참고 코드:\n{func_code}\n\n"),
    ("user", "### list_df:\n{list_df_text}\n\n")
]) | llm

response = chain.invoke({"func_code": func_code, "list_df_text": list_df_text}).content

attempt_count = 0
success = False

while attempt_count < 2:
    try:
        if "```python" in response:
            modified_code = response.split("```python")[-1].split("```")[0]
            print(f"[LOG] 실행할 코드:\n{modified_code}")
            exec(modified_code, globals())
            print(f"[Stage 1. 데이터 구조 파악] {attempt_count+1}차 시도 | ✅ 성공적으로 실행되었습니다!")
            success = True
            break
    except Exception as e:
        error_info = get_detailed_error_info(e)
        print(f"\n❌ {attempt_count+1}차 시도 실행 실패")
        print(f"🔍 에러 타입: {error_info['error_type']}")
        print(f"📝 에러 메시지: {error_info['error_message']}")
        print("\n🔍 스택 트레이스:")
        print(error_info['traceback'])

        if attempt_count == 0:
            print("\n🔄 에러 분석 및 코드 수정 요청 중...")
            
            # ✅ 에러 분석을 위한 프롬프트 템플릿
            ERROR_ANALYSIS_PROMPT = open(f'prompt/99.prompt_error_python.txt', 'r', encoding='utf-8').read().format(
                error_type= error_info['error_type'],
                error_message = error_info['error_message'],
                traceback = error_info['traceback'],
                context = error_info['context']
            )
            
            # 에러 분석 전문 프롬프트로 요청
            chain_error_analysis = ChatPromptTemplate.from_template(ERROR_ANALYSIS_PROMPT) | llm
            
            error_analysis_response = chain_error_analysis.invoke({
                
            })
            
            response = error_analysis_response.content
            print("\n✏️ LLM의 에러 분석 및 수정 제안:")
            print(response)
        else:
            print("\n❌ 최대 재시도 횟수 초과. 프로세스를 중단합니다.")
            
        attempt_count += 1

# ✅ Stage 1 결과 파일 로드 (엑셀)
print("\n🔄 Stage 1 결과 파일 로드 중...")
stage1_path = "../output/stage1/eda_summary.xlsx"
if os.path.exists(stage1_path):
    stage1_results = pd.read_excel(stage1_path, sheet_name=0)
    results["stage1"] = stage1_results
    print(f"✅ Stage 1 결과 로드 완료 (총 {len(stage1_results)} 개 행)")
else:
    print("❌ Stage 1 결과 파일이 존재하지 않습니다.")
    raise FileNotFoundError("Stage 1 결과 파일을 찾을 수 없습니다.")

# ✅ RAG 기반 컬럼 설명 추가
def load_vectorstore():
    if os.path.exists("./vectordb"):
        embeddings = OpenAIEmbeddings(
            model="text-embedding-3-large",
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        try:
            return FAISS.load_local("./vectordb", embeddings, allow_dangerous_deserialization=True)
        except Exception as e:
            print(f"⚠️ 벡터스토어 로드 중 오류 발생: {e}")
            return None
    return None

print("\n🔄 벡터스토어 로드 중...")
vectorstore = load_vectorstore()
if vectorstore:
    print("✅ 벡터스토어 로드 완료")
else:
    print("⚠️ 벡터스토어 로드 실패")

def search_column_descriptions(col_desc_df, vectorstore):
    """RAG를 사용하여 컬럼 설명을 검색하는 함수"""
    data = []
    df_after_llm = col_desc_df[['데이터프레임명', '컬럼명']]
    
    for i, row in df_after_llm.iterrows():
        table_name = row['데이터프레임명']
        col = row['컬럼명']
        
        search_query = f"테이블명 : {table_name} | 컬럼명 : {col}"
        docs = vectorstore.similarity_search(search_query, k=1)
        
        if docs:
            best_match = docs[0].page_content
            data.append({
                '데이터프레임명': table_name,
                '컬럼명': col,
                '컬럼설명': best_match.split('\n')[3].split('설명')[1].strip()
            })
        else:
            data.append({
                '데이터프레임명': table_name,
                '컬럼명': col,
                '컬럼설명': "설명 없음"
            })
    
    return pd.DataFrame(data)

print("\n🔄 RAG 기반 컬럼 설명 검색 시작...")
# ✅ Stage 2: RAG 기반 컬럼 설명 추가
rag_results = search_column_descriptions(stage1_results, vectorstore)
print(f"✅ RAG 검색 완료 (총 {len(rag_results)} 개 컬럼에 대한 설명 추가)")

print("\n🔄 최종 결과 통합 중...")
# ✅ Stage 1 결과와 RAG 결과를 통합
final_df = pd.merge(
    stage1_results,
    rag_results,
    on=['데이터프레임명', '컬럼명'],
    how='left'
)

# ✅ 최종 결과를 메모리 내 저장
results["stage2"] = final_df
print(f"✅ 최종 결과 통합 완료 (총 {len(final_df)} 개 행)")

print("\n✨ 모든 단계가 성공적으로 완료되었습니다.")

🔄 데이터 로드 시작...
✅ 'cust_enroll_history' 데이터 로드 완료
✅ 'cust_intg' 데이터 로드 완료
✅ 'product_info' 데이터 로드 완료
📊 총 3 개의 데이터프레임 로드 완료

✅ 프롬프트 및 함수 코드 로드 완료

🤖 LLM에 코드 생성 요청 중...


KeyError: "Input to ChatPromptTemplate is missing variables {'list_df_text'}.  Expected: ['func_code', 'list_df_text'] Received: ['func_code', 'list_df']\nNote: if you intended {list_df_text} to be part of the string and not a variable, please escape it with double curly braces like: '{{list_df_text}}'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT "